In [1]:
from gsmodutils import GSMProject
import cobra

In [2]:
project = GSMProject('.')  # load project from current directory

In [3]:
m = project.models[1]  # load E. coli GSM iJO1366 (https://www.embopress.org/doi/full/10.1038/msb.2011.65)

In [4]:
sol = m.optimize()  # maximise growth on glucose
print('growth rate (glc):',sol.objective_value)
project.save_conditions(m, 'glc_growth')

growth rate (glc): 0.9823718127269705


In [5]:
m.exchanges.get_by_id('EX_xyl__D_e').bounds = (-10,1000)  # allow xylose uptake
sol = m.optimize()
print('growth rate (glc+xyl):',sol.objective_value)
project.save_conditions(m, 'glc_and_xyl_growth')

growth rate (glc+xyl): 1.8015334467298303


In [6]:
m.exchanges.get_by_id('EX_glc__D_e').bounds = (0,1000)  # prevent glucose uptake
sol = m.optimize()
print('growth rate (xyl):',sol.objective_value)
project.save_conditions(m, 'xyl_growth')

growth rate (xyl): 0.8060965243972431


In [7]:
# Had to run the following command to get gsmodutils shell commands to work:
# set PATH=%PATH%;C:\Users\Admin\AppData\Roaming\Python\Python35\Scripts

In [8]:
# next we want to look at the pathways involved in xylose utilisation
# authors want "...a xylose catabolic pathway that directly flows into the 
# TCA cycle without interfering glycolysis and PPP..."
m_xyl = project.load_conditions('xyl_growth',m)

In [9]:
sol = m_xyl.optimize()

In [10]:
v = sol.fluxes

In [11]:
xyl_ppp_reactions = [
    'XYLI1',  # xylose isomerase
    'XYLK',  # xylose kinase
    
    #xylulose-5p and fructose-6p to ribose-5p
    'RPE',  # ribose phosphate isomerase
    'RPI',  # ribose phoshpate epimerase
    'GND',  #
    'PGL',  #
    'G6PDH2r',  #
    'PGI',  #
    
    #xylulose-5p and ribose-5p to g3p and erythrose-4p 
    'TKT1',  # transketolase 1
    'PFK_3',  # 
    'FBA3',  #
    'TPI',  #
    
    #xylulose-5p and erythrose-4p to g3p and fructose-6p
    'TKT2',  # transketolase 2
]

In [12]:
for reaction in xyl_ppp_reactions:
    print(reaction,v[reaction])

XYLI1 10.0
XYLK 10.0
RPE -1.828074279689817
RPI -4.992205592357372
GND 3.1962171787246763
PGL 3.1962171787246763
G6PDH2r 3.1962171787246763
PGI -3.1962171787246807
TKT1 4.239662090558438
PFK_3 0.0
FBA3 0.0
TPI 4.785635817711778
TKT2 3.932263629751745


In [15]:
#write essential pathway test, fluxes +/- 5%
required_fluxes={}
for reaction in xyl_ppp_reactions:
    reaction_flux = v[reaction]
    reaction_flux_margin = abs(reaction_flux*0.05)
    required_fluxes[reaction] = [reaction_flux-reaction_flux_margin,reaction_flux+reaction_flux_margin]
    #print(reaction,reaction_flux,reaction_flux_margin)
project.add_essential_pathway(
    'xylose_WT_PPP_2',
    description='WT pentose phosphate pathway flux on xylose',
    reactions=xyl_ppp_reactions,
    reaction_fluxes=required_fluxes,
    conditions=['xyl_growth'],
    models=['iJO1366.json']
)

In [ ]:

#flux = dict(PYR=[0.5, 1000]) # Required flux for a given reaction id
#project.add_essential_pathway('pathway_x', description='Example pathway', reactions=reactions, reaction_fluxes=flux)

In [18]:
print(m_xyl.reactions.EX_xyl__D_e,v['EX_xyl__D_e'])

EX_xyl__D_e: xyl__D_e <=>  -10.0


In [59]:
met = 'g3p_c'
[print(x,v[x.id]) for x in m.reactions if m.metabolites.get_by_id(met) in x.metabolites and v[x.id] != 0]

DXPS: g3p_c + h_c + pyr_c --> co2_c + dxyl5p_c 0.0022852836466661967
FBA: fdp_c <=> dhap_c + g3p_c 0.659880002629821
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c 13.66097720174148
TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c 4.239662090558454
TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c 3.932263629751758
TPI: dhap_c <=> g3p_c 4.785635817711799
TRPS3: 3ig3p_c --> g3p_c + indole_c 0.04582094473631282


[None, None, None, None, None, None, None]

In [45]:
met = 's17bp_c'
[print(x,v[x.id]) for x in m.reactions if m.metabolites.get_by_id(met) in x.metabolites and v[x.id] != 0]

FBA3: s17bp_c <=> dhap_c + e4p_c 4.239662090558454
PFK_3: atp_c + s7p_c --> adp_c + h_c + s17bp_c 4.239662090558454


[None, None]